# Projeto 2 - Classificador Automático de Sentimento

Integrantes:Lucas Leal Vale, Matheus Augusto Soares, Rafael Almada

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [2]:
df=pd.read_excel('tweets_riot.xlsx','Treinamento',sep=',')
dfTeste=pd.read_excel('teste.xlsx', 'Treinamento',sep=',')

In [3]:
lista1=['  ','   ',':','(',')','`','[',']','.','/','" ',"'"]
lista2=[' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ']
for i in range(len(lista1)):
    df['Treinamento']=df.Treinamento.str.replace(lista1[i], lista2[i])
df['Treinamento']=df.Treinamento.str.lower()

In [4]:
df['Treinamento']=df['Treinamento'].str.split()
df.head()

,Treinamento,Classificação,Legenda
0,"[n, é, pq, eu, tô, com, uma, crush, imensa, nã...",0,0 - IRRELEVANTE
1,"[rt, @favstommo, ai, cansei, de, viver, mas, n...",1,1 - RUIM
2,"[@carlosruaror, consegui,, riot, me, enviou, o...",2,2 - BOM
3,"[riot, fala, assim, mesmo, rs, https, t, co, r...",2,NaN
4,"[rt, @saradocinzas, aaaaaaaaaaaaa, vai, toma, ...",0,NaN


___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [5]:
dfLen=len(df)

dfBom=df[df['Classificação']==2]
dfBomLen=len(dfBom)

dfRuim=df[df['Classificação']==1]
dfRuimLen=len(dfRuim)

dfIrre=df[df['Classificação']==0]
dfIrreLen=len(dfIrre)
tw=dfBom["Treinamento"][2]


In [6]:
# # Probabilidades base
# ProbBom=len(dfBom)/(dfLen)
# ProbRuim=len(dfRuim)/(dfLen)
# ProbIrre=len(dfIrre)/(dfLen)
# print('A probablidade de ser Bom  é:{0}'.format(ProbBom))
# print('A probablidade de ser Ruim é:{0}'.format(ProbRuim))
# print('A probablidade de ser Irre é:{0}'.format(ProbIrre))


In [7]:
dfBomProb=dfBom["Treinamento"].sum()
dfBomProb=pd.Series(dfBomProb)
dfBomProb=dfBomProb.value_counts()
dfBomProb.head(5)

riot     79
a        67
co       62
t        62
https    62
dtype: int64

In [8]:
dfRuimProb=dfRuim["Treinamento"].sum()
dfRuimProb=pd.Series(dfRuimProb)
dfRuimProb=dfRuimProb.value_counts()
dfRuimProb.head(5)

riot    98
a       68
que     63
e       60
o       60
dtype: int64

In [9]:
dfIrreProb=dfIrre["Treinamento"].sum()
dfIrreProb=pd.Series(dfIrreProb)
dfIrreProb=dfIrreProb.value_counts()
dfIrreProb.head(5)

de     49
rt     41
que    37
o      29
eu     27
dtype: int64

In [10]:
dfTotal=df.Treinamento.sum()
dfTotal=pd.Series(dfTotal)
dfTotal=dfTotal.value_counts()
dfTotal.head()

riot    190
a       160
de      143
que     129
e       109
dtype: int64

In [11]:
dfPalavras=df.Treinamento.sum()
dfPalavras=pd.Series(dfPalavras)
dfPalavras=len(dfPalavras)
dfPalavras

5523

In [12]:
BoasPalavras=dfBom["Treinamento"].sum()
BoasPalavras=pd.Series(BoasPalavras)
BoasPalavras=len(BoasPalavras)
BoasPalavras

1585

In [13]:
RuimPalavras=dfRuim["Treinamento"].sum()
RuimPalavras=pd.Series(RuimPalavras)
RuimPalavras=len(RuimPalavras)
RuimPalavras

2184

In [14]:
IrrePalavras=dfIrre["Treinamento"].sum()
IrrePalavras=pd.Series(IrrePalavras)
IrrePalavras=len(IrrePalavras)
IrrePalavras

1754

In [15]:
# Probabilidades base
ProbBom=BoasPalavras/(dfPalavras)
ProbRuim=RuimPalavras/(dfPalavras)
ProbIrre=IrrePalavras/(dfPalavras)
print('A probablidade de ser Bom  é:{0}'.format(ProbBom))
print('A probablidade de ser Ruim é:{0}'.format(ProbRuim))
print('A probablidade de ser Irre é:{0}'.format(ProbIrre))

A probablidade de ser Bom  é:0.2869817128372261
A probablidade de ser Ruim é:0.39543726235741444
A probablidade de ser Irre é:0.3175810248053594


In [16]:
#Calculando as probabilidades base
#Prob palavra/bom
dfBomProb2=(dfBomProb+1)/(len(dfBomProb)+len(dfTotal))
#Prob palavra/ruim
dfRuimProb2=(dfRuimProb+1)/(len(dfRuimProb)+len(dfTotal))
#Prob palavra/irrelevante
dfIrreProb2=(dfIrreProb+1)/(len(dfIrreProb)+len(dfTotal))
dfBomProb2.head(5)

riot     0.033571
a        0.028535
co       0.026437
t        0.026437
https    0.026437
dtype: float64

In [17]:
# for i in df.Treinamento:
#     for u in i:
#         print("u: ", u)
#         print("dft u:", dfTotal[u])


In [18]:
resultado=[]

In [19]:
def bayes(df,dfPalavras,lista):
  
    laplace=(1/(dfPalavras))# obs da folha
    for i in df: # for i in df.index:
        c_bom=1
        c_ruim=1
        c_irre=1
        for u in i:
            #p=dfTotal[u]/len(dfTotal) #(Probabilidade da palavra no espaço amostral de todas as palavras)
            if u not in dfBomProb2:
                p_bom=laplace
            else:
                p_bom=dfBomProb2[u]
            if u not in dfRuimProb2:
                p_ruim=laplace
            else:
                p_ruim=dfRuimProb2[u]
            if u not in dfIrreProb2:
                p_irre=laplace
            else:
                p_irre=dfIrreProb2[u]
                
            c_bom*=(p_bom)
            c_ruim*=(p_ruim)
            c_irre*=(p_irre)
        c_bom=c_bom*(ProbBom)
        #print("c_bom:",c_bom)
        c_ruim=c_ruim*(ProbRuim)
        #print("c_ruim:",c_ruim)
        c_irre=c_irre*(ProbIrre)
        #print("c_irre:",c_irre)

        if c_bom>c_ruim and c_bom>c_irre:
            #Classifica como Bom (2)
            lista.append(2)

        elif c_ruim>c_bom and c_ruim>c_irre:
            #Classifica como Ruim (1)
            lista.append(1)
            
        else:# c_irre>c_bom and c_irre>c_ruim:
            #Classifica como Irrelevante (0) 
            lista.append(0)
    return lista

In [20]:
a=bayes(df["Treinamento"],dfPalavras,resultado)

In [21]:
print(len(resultado))
print(a)

300
[0, 1, 2, 2, 0, 0, 0, 2, 0, 2, 2, 2, 1, 1, 2, 0, 2, 0, 2, 1, 1, 0, 0, 2, 0, 1, 2, 1, 0, 2, 1, 1, 0, 1, 2, 0, 0, 0, 2, 1, 0, 2, 1, 1, 2, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 2, 1, 1, 0, 0, 0, 0, 2, 0, 0, 1, 2, 1, 2, 2, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 2, 1, 0, 0, 2, 1, 0, 2, 2, 2, 0, 1, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 2, 1, 1, 2, 1, 1, 0, 1, 0, 1, 2, 0, 0, 2, 0, 1, 1, 1, 0, 1, 2, 2, 2, 0, 2, 2, 0, 1, 0, 0, 2, 2, 2, 0, 1, 1, 0, 0, 0, 0, 2, 1, 2, 0, 1, 1, 1, 1, 1, 1, 2, 2, 1, 0, 1, 2, 1, 1, 1, 0, 0, 1, 2, 1, 1, 2, 1, 0, 1, 2, 2, 2, 1, 0, 0, 2, 1, 2, 0, 1, 0, 1, 1, 2, 0, 2, 0, 2, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 2, 1, 2, 2, 0, 2, 0, 1, 1, 1, 0, 1, 2, 0, 1, 0, 0, 0, 2, 2, 2, 2, 0, 0, 1, 0, 0, 0, 2, 0, 0, 2, 1, 1, 2, 0, 2, 1, 2, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 2, 0, 0, 1, 0, 2, 1, 0, 2, 0, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 0, 2, 2, 2, 1, 1, 2, 0, 2, 2, 0, 2, 1, 1, 1, 2, 2, 0, 1, 2, 1, 2, 2, 1, 0, 1]


In [22]:
df["Resultado"]=a

In [23]:
df.head()

,Treinamento,Classificação,Legenda,Resultado
0,"[n, é, pq, eu, tô, com, uma, crush, imensa, nã...",0,0 - IRRELEVANTE,0
1,"[rt, @favstommo, ai, cansei, de, viver, mas, n...",1,1 - RUIM,1
2,"[@carlosruaror, consegui,, riot, me, enviou, o...",2,2 - BOM,2
3,"[riot, fala, assim, mesmo, rs, https, t, co, r...",2,NaN,2
4,"[rt, @saradocinzas, aaaaaaaaaaaaa, vai, toma, ...",0,NaN,0


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [24]:
r=[]
b=bayes(dfTeste['Treinamento'],dfPalavras,r)
dfTeste['Resultado']=b

In [25]:
dfTeste

,Treinamento,Classificação,Legenda,Resultado
0,gostei de um vídeo @youtube https://t.co/4oilk...,2.0,0 - IRRELEVANTE,0
1,"rt @lolesportsbr: ""senhor do meu destino mano,...",1.0,1 - RUIM,0
2,"rt @ahsbrasil: sinopse ep. 01 - ""the end""\n\nn...",0.0,2 - BOM,0
3,"@yetz1 riot ta de sacanagem só pode, nunca viu...",2.0,NaN,1
4,"@guilhermdw vc não tá entendendo, não é uma va...",2.0,NaN,0
5,riot cade meu rp,2.0,NaN,1
6,@giogrosso_dsr é o mid season invitational. ac...,2.0,NaN,0
7,riot muito sensata https://t.co/aixjlyvpyg,2.0,NaN,2
8,"rt @yetz1: riot pensa em tanta coisa, pq não p...",2.0,NaN,0
9,riot eu to tentando parar de jogar pfvr colabo...,2.0,NaN,1


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
